In [1]:
%load_ext autoreload
import os 
import random
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import numpy as np
import DocumentClassifierUsingBertJapanese as clf
%autoreload

In [2]:
df = pd.read_csv("./data/train_eval.tsv", sep='\t')
# print(df)

le = LabelEncoder()
# df['label'] = le.fit_transform(df.label.values)
# df = df[['text', 'label']]
df.columns = ['Text', 'Label']
train_df, val_df = train_test_split(df, test_size=0.2, stratify=df.Label)
df.head()

,Text,Label
0,待望の…待望のUSB経由のネットワーク接続が可能に！！Android情報サイト「AppCom...,3
1,GALAXY SIII SC-06Dが発表！NTTドコモは16日、今夏に発売する予定の新モデ...,3
2,女子サッカーアメリカ代表のFWアレックス・モーガン（22歳）が、米誌『スポーツ・イラストレイ...,2
3,プロボクサー・亀田興毅が、かねてから語っていた“30歳で引退”という目指すべき現役のゴールを...,2
4,GALAXY SII SC-02CにAndroid 4.0 ICSが提供開始！NTTドコモは...,3


In [3]:
model = clf.DocumentClassifier(num_labels=9, num_epochs=100)

Some weights of the model checkpoint at cl-tohoku/bert-base-japanese-whole-word-masking were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialize

In [37]:
model.fit(train_df, val_df, early_stopping_rounds=10)

[I 210328 12:52:15 DocumentClassifierUsingBertJapanese:172] [Start]Create DataSets from DataFrames
[I 210328 12:52:45 DocumentClassifierUsingBertJapanese:175] [Finished]Create DataSets from DataFrames
[I 210328 12:52:45 DocumentClassifierUsingBertJapanese:179] [Start]Create DataLoaders
[I 210328 12:52:45 DocumentClassifierUsingBertJapanese:182] [Finished]Create DataLoaders
[I 210328 12:52:45 DocumentClassifierUsingBertJapanese:227] 使用デバイス：cuda:0
[I 210328 12:52:45 DocumentClassifierUsingBertJapanese:228] -----start-------


TypeError: max() received an invalid combination of arguments - got (str, int), but expected one of:
 * (Tensor input)
 * (Tensor input, name dim, bool keepdim, *, tuple of Tensors out)
 * (Tensor input, Tensor other, *, Tensor out)
 * (Tensor input, int dim, bool keepdim, *, tuple of Tensors out)


In [33]:
y_proba = model.predict(val_df)
y_proba

[I 210328 12:44:21 DocumentClassifierUsingBertJapanese:329] [Start]Create DataSet, DataLoader from DataFrame
[I 210328 12:44:27 DocumentClassifierUsingBertJapanese:334] [Finished]Create DataSet, DataLoader from DataFrame
[I 210328 12:44:27 DocumentClassifierUsingBertJapanese:337] 使用デバイス：cuda:0
[I 210328 12:44:27 DocumentClassifierUsingBertJapanese:338] -----start-------
100%|██████████| 37/37 [01:56<00:00,  3.14s/it]
[I 210328 12:46:23 DocumentClassifierUsingBertJapanese:348] -----finished-------


array([[0.10159193, 0.12381991, 0.07763703, ..., 0.12575388, 0.11685082,
        0.14893092],
       [0.09534844, 0.1253567 , 0.10198842, ..., 0.10485069, 0.13035609,
        0.15408024],
       [0.10034444, 0.11472436, 0.11446261, ..., 0.10865485, 0.10873953,
        0.14498372],
       ...,
       [0.10490467, 0.11947428, 0.08808235, ..., 0.11664749, 0.10675719,
        0.14918758],
       [0.10365929, 0.12079606, 0.0910448 , ..., 0.1206104 , 0.10771292,
        0.14538683],
       [0.10329206, 0.11612666, 0.09550446, ..., 0.12401586, 0.11218169,
        0.1439864 ]], dtype=float32)

In [42]:
print(y_proba.shape)
print(y_proba.argmax(axis=1))

(1181, 9)
[8 8 5 ... 5 5 5]


___

In [4]:
import torch

In [5]:
text = '私は、火星という星まで宇宙旅行してみたいです。'
tokenized_text = model.tokenizer.tokenize(text)
print(tokenized_text)

#ここでは「火星」をMASKして、「火星」以外の単語を探そうとしている
masked_index = 3
tokenized_text[masked_index] = '[MASK]'

# Convert token to vocabulary indices
indexed_tokens = model.tokenizer.convert_tokens_to_ids(tokenized_text)
print(indexed_tokens)

tokens_tensor = torch.tensor([indexed_tokens])
print(tokens_tensor)


['私', 'は', '、', '火星', 'という', '星', 'まで', '宇宙', '旅行', 'し', 'て', 'みたい', 'です', '。']
[1325, 9, 6, 4, 140, 1414, 126, 1669, 3807, 15, 16, 17131, 2992, 8]
tensor([[ 1325,     9,     6,     4,   140,  1414,   126,  1669,  3807,    15,
            16, 17131,  2992,     8]])


In [6]:
from transformers import AutoTokenizer, AutoModelWithLMHead

In [7]:
model_auto = AutoModelWithLMHead.from_pretrained("cl-tohoku/bert-base-japanese-whole-word-masking")
model.eval()


/usr/local/lib/python3.6/dist-packages/transformers/models/auto/modeling_auto.py:1010: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  FutureWarning,
Some weights of the model checkpoint at cl-tohoku/bert-base-japanese-whole-word-masking were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a B

AttributeError: 'DocumentClassifier' object has no attribute 'eval'

In [ ]:

# Predict
with torch.no_grad():
    outputs = model(tokens_tensor)
    #データ構成 tapleの1個目固定、 [1, 14, 32000] 1個の文章、14つの単語、32000次元の数値
    predictions = outputs[0][0, masked_index].topk(7) # 予測結果の上位7件を抽出

print(outputs[0].shape)
print(predictions.indices)

#「火星」と置き換える文字候補を出す
for i, index_t in enumerate(predictions.indices):
    index = index_t.item()

    #数値を元の単語に戻す
    token = model.tokenizer.convert_ids_to_tokens([index ])[0]
    print(i, token)

    tokenized_text[masked_index] = token
    print("".join(tokenized_text))

#たしかにそれらしい文章が出力されました。


In [1]:
import sys
import pandas as pd
import sentencepiece as spm
import logging
import numpy as np

from keras import utils
from keras.models import load_model
from keras.preprocessing.sequence import pad_sequences
from keras_bert import load_trained_model_from_checkpoint
from keras_bert import get_custom_objects
from sklearn.metrics import classification_report, confusion_matrix


sys.path.append('modules')

# SentencePieceProccerモデルの読込
spp = spm.SentencePieceProcessor()
spp.Load('/content/drive/My Drive/bert/bert-wiki-ja/wiki-ja.model')
# BERTの学習したモデルの読込
model_filename = '/content/drive/My Drive/bert/models/knbc_finetuning.model'
model = load_model(model_filename, custom_objects=get_custom_objects())

SEQ_LEN = 103
maxlen = SEQ_LEN

def _get_indice(feature):
    indices = np.zeros((maxlen), dtype=np.int32)

    tokens = []
    tokens.append('[CLS]')
    tokens.extend(spp.encode_as_pieces(feature))
    tokens.append('[SEP]')

    for t, token in enumerate(tokens):
        if t >= maxlen:
            break
        try:
            indices[t] = spp.piece_to_id(token)
        except:
            logging.warn('unknown')
            indices[t] = spp.piece_to_id('<unk>')
    return indices

feature = "昨日は携帯電話を買いに行った。"

test_features = []
test_features.append(_get_indice(feature))
test_segments = np.zeros(
    (len(test_features), maxlen), dtype=np.float32)

predicted_test_labels = model.predict(
    [test_features, test_segments]).argmax(axis=1)

label_data = pd.read_csv('/content/drive/My Drive/bert/label_id/id_category.csv')
label = label_data.query(f'id == {predicted_test_labels[0]}')
label = label.iloc[0]
label_name = label['label']
print(label_name)

ModuleNotFoundError: No module named 'keras_bert'